In [ ]:
# # ユーザー認証
# from google.colab import auth
# auth.authenticate_user()

#Google setting
from google import colab
colab.drive.mount('/content/gdrive')

#Directory setting
b_dir='./gdrive/My Drive/AI_Seminar/NLP_1/' # Setting working directory
dataset_dir=b_dir+'dataset/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%%bash

free -h

# ツールのダウンロード
apt-get update
apt-get install mecab file swig libmecab-dev mecab-ipadic-utf8
pip install mecab-python3==0.996.5
pip install transformers==2.11.0
pip install tqdm
pip install category_encoders

              total        used        free      shared  buff/cache   available
Mem:            12G        577M        9.8G        1.0M        2.4G         11G
Swap:            0B          0B          0B
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Re

In [ ]:
# insall
import numpy as np
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from scipy import stats
from sklearn.model_selection import KFold
import os,urllib,glob,tarfile
from transformers import BertJapaneseTokenizer, BertModel, BertConfig, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
import category_encoders as ce

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# データの読み込み
# train_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1mYL2JNkfIPRvvDdw8m0Fnh8mKYb4ND5S")  # train.csv
# valid_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1Mdyv3DPcb1AncxE7H739QWyPkMqcdjqI")  # dev.csv
test_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1yHKfDrs6ZAx4fizToTF7GL7ROvGk7Ry5")  # test.csv
# train_df = pd.read_csv("./dataset/train.csv")
# val_df = pd.read_csv("./dataset/dev.csv")
# test_submission = pd.read_csv("./dataset/test.csv")

# train_df.head(5)

,movieName,title,text,rating
0,HERO,楽しみにしてるよ～～!!!,この前、HEROを１話から見直しました!!もぅドキドキで楽しいっ!!あたしも、西遊記見たとき...,5
1,インセプション,期待しすぎた…,DVDでの鑑賞。\nちょっと観て時間がたったので、ざっくりレビューです。\n\n根本的なアイ...,3
2,パッチギ！ LOVE&PEACE,真面目に生きている在日の人たちが可哀想,こんな変な映画が、全国上映されるからまた、在日批判が出てくるんじゃないの？,1
3,ダ・ヴィンチ・コード,いまいちの映画でした,はっきり言って期待しすぎたのが悪かったのか全く面白くなかったです。この手の映画は宗教がらみな...,2
4,踊る大捜査線 THE MOVIE 3 ヤツらを解放せよ！,残念なチャリティーマッチ,制作側の意図や事情を知らずにむやみに酷評するのははばかられますが、はっきり言ってつまらなかっ...,2


In [ ]:
list_cols = ['movieName']
ce_oe = ce.OrdinalEncoder(cols=list_cols, handle_unknown='impute')
test_df = ce_oe.fit_transform(test_df)

In [ ]:
test_df['text'] = test_df['title'].str.cat(test_df['text'])

In [ ]:
test_df['text'] = test_df['movieName'].str.cat(test_df['text'])

In [ ]:
# tokenizerの定義とvocabを作成
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
pad = tokenizer.vocab["[PAD]"]
# max_lengths=256
max_lengths=512

In [ ]:
# 上記のinput_dataの作成方法をもとにdataset化
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels=[]):
        self.input_ids, self.segment_ids, self.attention_masks = [],[],[]
        for text in tqdm(texts):
            token = ["[CLS]"]+tokenizer.tokenize(text)[:max_lengths-2]+["[SEP]"]
            input_id = tokenizer.convert_tokens_to_ids(token)
            segment_id = [0]*max_lengths
            attention_mask = [1]*len(input_id)+[0]*(max_lengths - len(input_id))
            input_id = input_id+[pad]*(max_lengths-len(input_id))
            self.input_ids.append(input_id)
            self.segment_ids.append(segment_id)
            self.attention_masks.append(attention_mask)
        self.input_ids = np.array(self.input_ids)
        self.segment_ids = np.array(self.segment_ids)
        self.attention_masks = np.array(self.attention_masks)
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if len(self.labels):
            return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx], self.labels[idx]
        else:
            return self.input_ids[idx], self.segment_ids[idx], self.attention_masks[idx]

In [ ]:
# # datasetとdataloaderの作成
# batch_size = 8  # 8
# # X, y = train_df.drop("rating").values, train_df["rating"].values-1
# X, y = train_df["text"].values, train_df["rating"].values-1
# # X, y = train_df["title"].values, train_df[["rating_1", "rating_2", "rating_3", "rating_4", "rating_5"]].values
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=100,stratify=y)
# # X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=100,stratify=y)
# train_ds = ReviewDataset(texts=X_train, labels=y_train)
# train_dl = torch.utils.data.DataLoader(
#     train_ds, batch_size=batch_size, shuffle=True)
# X_validOrg, y_validOrg = valid_df["text"].values, valid_df["rating"].values-1
# X_valid, y_valid = np.concatenate([X_val, X_validOrg]), np.concatenate([y_val, y_validOrg])
# X_valid, _, y_valid, _ = train_test_split(X_valid, y_valid, test_size=0.64, random_state=100, stratify=y_valid)
# valid_ds = ReviewDataset(texts=X_valid, labels=y_valid)
# valid_dl = torch.utils.data.DataLoader(
#     valid_ds, batch_size=batch_size, shuffle=False)

In [ ]:
# Bertの事前学習済みエンコーダーを使用して5値クラス分類器を作成

class BertClassification(nn.Module):
    def __init__(self, bert, n_classes=2):
        super(BertClassification, self).__init__()

        # BERTモジュール
        self.bert = bert  # BERTモデル

        self.cls = nn.Linear(in_features=768, n_classes)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        pooled_output = self.bert(
                  input_ids, token_type_ids, attention_mask)[1]

        # 入力文章の1単語目[CLS]の特徴量を使用して、ポジ・ネガを分類します
        pooled_output = pooled_output.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        outputs = self.cls(pooled_output)

        return outputs


In [ ]:
cuda = torch.cuda.is_available()
# 学習済みモデルをダウンロード
bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertClassification(bert, n_classes=5)
# 高速化
torch.backends.cudnn.benchmark = True
if cuda:
    model.cuda()
# optimizerの設定
# optimizer = optim.Adam(model.parameters(),lr = 4e-4 ,betas=(0.9, 0.999))
optimizer = optim.AdamW(model.parameters(),lr = 1e-5 ,betas=(0.9, 0.999))
# optimizer = optim.AdamW(model.parameters(), lr = 1e-4)
# scheduler = get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=(len(X_train) // batch_size + 1) * 1,
#     num_training_steps=(len(X_train) // batch_size + 1) * 5
# )

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

In [ ]:
model.load_state_dict(torch.load(dataset_dir+'model_valid.hdf5'))

<All keys matched successfully>

In [ ]:
# test datset dataloaderを作成。
test_ds = ReviewDataset(texts=test_df["text"].values)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=1, shuffle=False)

In [ ]:
model.eval()
if cuda:
    model.cuda()
else:
    model.cpu()

# 予測を付与。
lists_proba = []
lists = []
for input_ids, segment_ids, attention_masks in tqdm(test_dl):
    if cuda:
        input_ids, segment_ids, attention_masks =\
        input_ids.cuda(), segment_ids.cuda(), attention_masks.cuda()
    outputs = model(input_ids = input_ids,
                  token_type_ids = segment_ids,
                  attention_mask = attention_masks)
    pred_proba = outputs.softmax(dim=1)
    pred = torch.argmax(pred_proba, axis=1)
    lists += list(pred.cpu().detach().numpy()+1) 
    lists_proba += list(pred_proba.cpu().detach().numpy()) 

In [ ]:
# test_df["Label"]=lists
# submission_df=test_df[["Id","Label"]]
# submission_df.to_csv("sample_submission.csv",index=False)

In [ ]:
result = "\n".join([str(rating) for rating in lists])
open(dataset_dir+"test_pred_bert_first.txt", "w").write(result)
pd.DataFrame(lists_proba).to_csv(dataset_dir+"bert_pred_proba_first.csv")

65353